# **Instructions**

Pour générer les fichiers train/test, il suffit de changer la valeur des booleens, **de se placer dans cette cellule** et choisir:

Execution -> Executer à la suite

In [985]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from sklearn.preprocessing import LabelEncoder

In [986]:
generateTrain = True
generateTest = False

if generateTrain == generateTest:
  print("Can't generate both of them at the same time, generating Train..")
  generateTrain = True
  generateTest = False

if generateTrain :
  df_train = pd.read_csv("train.csv")
  df_train_copy = df_train.copy()


if generateTest :
  df_train = pd.read_csv("test.csv")
  df_train_copy = df_train.copy()

In [987]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7500 entries, 0 to 7499
Data columns (total 24 columns):
 #   Column                                       Non-Null Count  Dtype  
---  ------                                       --------------  -----  
 0   Id                                           7500 non-null   object 
 1   User Name                                    7500 non-null   object 
 2   Personal URL                                 3256 non-null   object 
 3   Profile Cover Image Status                   7410 non-null   object 
 4   Profile Verification Status                  7500 non-null   object 
 5   Profile Text Color                           7434 non-null   object 
 6   Profile Page Color                           7422 non-null   object 
 7   Profile Theme Color                          7428 non-null   object 
 8   Is Profile View Size Customized?             7500 non-null   bool   
 9   UTC Offset                                   7014 non-null   float64
 10  

**Dropping classes**

In [988]:
#df_train.drop(labels=["User Name", "Personal URL", "Location", "User Time Zone"], axis=1, inplace=True)
df_train.drop(labels=["User Name", "Location", "User Time Zone"], axis=1, inplace=True)

**Converting URL to binary**

In [989]:
df_train["Personal URL"] = df_train['Personal URL'].replace('.*', 1, regex=True)
df_train["Personal URL"] = df_train['Personal URL'].replace(np.nan, 0, regex=True)

df_train

,Id,Personal URL,Profile Cover Image Status,Profile Verification Status,Profile Text Color,Profile Page Color,Profile Theme Color,Is Profile View Size Customized?,UTC Offset,Location Public Visibility,User Language,Profile Creation Timestamp,Num of Followers,Num of People Following,Num of Status Updates,Num of Direct Messages,Profile Category,Avg Daily Profile Visit Duration in seconds,Avg Daily Profile Clicks,Profile Image,Num of Profile Likes
0,AL85S14OMDPF01I9,0.0,Set,Verified,db1a2c,eaf0f2,e70409,False,39600.0,Enabled,en,Thu Nov 27 05:24:59 +0000 2008,95763,4289,30809,873,business,14.792,1.5761,AL85S14OMDPF01I9.png,2815
1,HI11QOPD7BLJTO7Q,0.0,Set,Verified,0099cc,f6ffd1,fff04d,False,NaN,Enabled,en,Fri Jan 15 18:00:46 +0000 2010,1018746,289,8150,290,unknown,8.183,11.2782,HI11QOPD7BLJTO7Q.png,1242
2,JS49LP5P72RI1OQB,0.0,Set,Not verified,1fc2de,efefef,1fc2de,False,-18000.0,Enabled,en,Fri Oct 02 20:15:06 +0000 2009,13444,1876,4698,227,unknown,31.823,0.5725,JS49LP5P72RI1OQB.png,1559
3,S0GDSC09MACCLBJP,1.0,Not set,Verified,050000,616161,00090a,False,-14400.0,Enabled,en,Thu Feb 19 14:37:22 +0000 2009,339168,1148,53216,4035,business,23.052,4.0265,S0GDSC09MACCLBJP.png,6342
4,CRSEMK4QER6LDJSA,1.0,Set,Not verified,58424d,f7f7f7,000000,False,-18000.0,Enabled,en,Tue Mar 31 13:27:52 +0000 2009,9215,93,3271,130,unknown,8.418,3.9229,CRSEMK4QER6LDJSA.png,1078
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7495,KP5DR7CTN8CNLG8P,0.0,Not set,Verified,490707,ddeef6,000000,False,-25200.0,Disabled,en,Thu Jan 20 20:22:20 +0000 2011,6086,76,985,56,unknown,24.792,8.1106,KP5DR7CTN8CNLG8P.png,1373
7496,SGNF9P1P3922T34T,0.0,Not set,Not verified,000000,ffffff,ffffff,False,-10800.0,Enabled,pt,Wed Mar 16 03:50:49 +0000 2011,280190,38604,20652,172,unknown,26.006,0.8956,SGNF9P1P3922T34T.png,1407
7497,O00FTLDT6O803LD0,0.0,Set,Verified,0084b4,ddffcc,00133b,False,7200.0,enabled,en,Sat Mar 14 16:18:06 +0000 2009,21647,49,2348,271,,15.030,2.8867,O00FTLDT6O803LD0.png,1235
7498,9F04JLR9JGT8R3EJ,0.0,Set,Not verified,91d2fa,ddffcc,fa743e,False,-10800.0,enabled,pt,Sat Jul 25 16:08:25 +0000 2009,89005,256,73830,346,business,11.948,1.7756,9F04JLR9JGT8R3EJ.png,1105


**Managing TimeStamp**

In [990]:
trips_start = df_train[["Id", "Profile Creation Timestamp"]].copy()
trips_start["Profile Creation Timestamp"] = pd.to_datetime(trips_start["Profile Creation Timestamp"])

#https://www.w3resource.com/python-exercises/pandas/datetime/pandas-datetime-exercise-8.php
trips_start["Profile Creation Day"] = trips_start["Profile Creation Timestamp"].dt.day
trips_start["Profile Creation WeekDay"] = trips_start["Profile Creation Timestamp"].dt.day_name()
trips_start["Profile Creation Month"] = trips_start["Profile Creation Timestamp"].dt.month
trips_start["Profile Creation Year"] = trips_start["Profile Creation Timestamp"].dt.year
trips_start["Profile Creation Hour"] = trips_start["Profile Creation Timestamp"].dt.hour
trips_start = trips_start.drop(columns=["Profile Creation Timestamp"])

df_train = df_train.join(trips_start.set_index('Id'), on='Id')

#Here, this is an totally optional step. I moved the 3 features added to the dataframe to the front of the dataframe to make it easy to see them when printing
#https://stackoverflow.com/questions/25122099/move-column-by-name-to-front-of-table-in-pandas
for col_name in ["Profile Creation Hour", "Profile Creation WeekDay", "Profile Creation Day", "Profile Creation Month", "Profile Creation Year"]:
  mid = df_train[col_name]
  df_train.drop(labels=col_name, axis=1, inplace = True)
  df_train.insert(10, col_name, mid)

df_train.drop(columns=["Profile Creation Timestamp"], inplace=True)

**Converting Categories to numeric**

In [991]:
cover_encoder = LabelEncoder()
df_train['Profile Cover Image Status'] = df_train['Profile Cover Image Status'].replace(np.nan, "unknown", regex=True)
df_train['Profile Cover Image Status'] = cover_encoder.fit_transform(df_train['Profile Cover Image Status'])
print(cover_encoder.classes_)

location_encoder = LabelEncoder()
df_train["Location Public Visibility"] = df_train['Location Public Visibility'].replace(np.nan, '??', regex=True)
df_train['Location Public Visibility'] = df_train['Location Public Visibility'].str.lower()
df_train["Location Public Visibility"] = location_encoder.fit_transform(df_train["Location Public Visibility"])
print( location_encoder.classes_)

view_size_encoder = LabelEncoder()
df_train["Is Profile View Size Customized?"] = view_size_encoder.fit_transform(df_train["Is Profile View Size Customized?"])
print( view_size_encoder.classes_)

language_encoder = LabelEncoder()
df_train["User Language"] = df_train['User Language'].str.lower()
df_train["User Language"] = language_encoder.fit_transform(df_train["User Language"])
print( language_encoder.classes_)

category_encoder = LabelEncoder()
df_train["Profile Category"] = df_train['Profile Category'].replace(np.nan, 'unknown', regex=True)
df_train["Profile Category"] = df_train['Profile Category'].replace(' ', 'unknown', regex=True)
df_train['Profile Category'] = df_train['Profile Category'].str.lower()

df_train["Profile Creation WeekDay"] = df_train["Profile Creation WeekDay"].map({'Monday':0, 'Tuesday':1, 'Wenesday':2, 'Thursday':3, "Friday":4, "Saturday":5, "Sunday":6})
in_order_weekdays = ["Monday", "Tuesday", "Wenesday", "Thursday", "Friday", "Saturday", "Sunday"]
print(in_order_weekdays)



['Not set' 'Set' 'unknown']
['??' 'disabled' 'enabled']
[False  True]
['ar' 'ca' 'cs' 'da' 'de' 'el' 'en' 'en-gb' 'es' 'fi' 'fr' 'hu' 'id' 'it'
 'ja' 'ko' 'nl' 'pl' 'pt' 'ru' 'sk' 'sr' 'sv' 'th' 'tr' 'uk' 'zh-cn'
 'zh-tw']
['Monday', 'Tuesday', 'Wenesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']


In [992]:
df_train

,Id,Personal URL,Profile Cover Image Status,Profile Verification Status,Profile Text Color,Profile Page Color,Profile Theme Color,Is Profile View Size Customized?,UTC Offset,Location Public Visibility,Profile Creation Year,Profile Creation Month,Profile Creation Day,Profile Creation WeekDay,Profile Creation Hour,User Language,Num of Followers,Num of People Following,Num of Status Updates,Num of Direct Messages,Profile Category,Avg Daily Profile Visit Duration in seconds,Avg Daily Profile Clicks,Profile Image,Num of Profile Likes
0,AL85S14OMDPF01I9,0.0,1,Verified,db1a2c,eaf0f2,e70409,0,39600.0,2,2008,11,27,3.0,5,6,95763,4289,30809,873,business,14.792,1.5761,AL85S14OMDPF01I9.png,2815
1,HI11QOPD7BLJTO7Q,0.0,1,Verified,0099cc,f6ffd1,fff04d,0,NaN,2,2010,1,15,4.0,18,6,1018746,289,8150,290,unknown,8.183,11.2782,HI11QOPD7BLJTO7Q.png,1242
2,JS49LP5P72RI1OQB,0.0,1,Not verified,1fc2de,efefef,1fc2de,0,-18000.0,2,2009,10,2,4.0,20,6,13444,1876,4698,227,unknown,31.823,0.5725,JS49LP5P72RI1OQB.png,1559
3,S0GDSC09MACCLBJP,1.0,0,Verified,050000,616161,00090a,0,-14400.0,2,2009,2,19,3.0,14,6,339168,1148,53216,4035,business,23.052,4.0265,S0GDSC09MACCLBJP.png,6342
4,CRSEMK4QER6LDJSA,1.0,1,Not verified,58424d,f7f7f7,000000,0,-18000.0,2,2009,3,31,1.0,13,6,9215,93,3271,130,unknown,8.418,3.9229,CRSEMK4QER6LDJSA.png,1078
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7495,KP5DR7CTN8CNLG8P,0.0,0,Verified,490707,ddeef6,000000,0,-25200.0,1,2011,1,20,3.0,20,6,6086,76,985,56,unknown,24.792,8.1106,KP5DR7CTN8CNLG8P.png,1373
7496,SGNF9P1P3922T34T,0.0,0,Not verified,000000,ffffff,ffffff,0,-10800.0,2,2011,3,16,NaN,3,18,280190,38604,20652,172,unknown,26.006,0.8956,SGNF9P1P3922T34T.png,1407
7497,O00FTLDT6O803LD0,0.0,1,Verified,0084b4,ddffcc,00133b,0,7200.0,2,2009,3,14,5.0,16,6,21647,49,2348,271,unknown,15.030,2.8867,O00FTLDT6O803LD0.png,1235
7498,9F04JLR9JGT8R3EJ,0.0,1,Not verified,91d2fa,ddffcc,fa743e,0,-10800.0,2,2009,7,25,5.0,16,18,89005,256,73830,346,business,11.948,1.7756,9F04JLR9JGT8R3EJ.png,1105


**Converting colors to numeric**

In [993]:
def hex2decimal(hex_value):

  if len(str(hex_value)) == 6:
    hex = str(hex_value)
    r = "{:.2f}".format(float(int(hex[0:2], 16))/255.0)
    g = "{:.2f}".format(float(int(hex[2:4], 16))/255.0)
    b = "{:.2f}".format(float(int(hex[4:6], 16))/255.0)

    return str(r) + "|" + str(g) + "|" + str(b)
  else:
    return hex_value



In [994]:
df_train['Profile Text Color'] = df_train['Profile Text Color'].apply(lambda x: x if(len(str(x)) == 6) else np.nan)
df_train['Profile Text Color'] = df_train['Profile Text Color'].apply(lambda x: hex2decimal(x))
new_cols = df_train['Profile Text Color'].str.split("|", expand = True)

df_train.insert(df_train.columns.get_loc("Profile Text Color"), "Profile Text Color R", new_cols[0])
df_train.insert(df_train.columns.get_loc("Profile Text Color"), "Profile Text Color G", new_cols[1])
df_train.insert(df_train.columns.get_loc("Profile Text Color"), "Profile Text Color B", new_cols[2])

df_train.drop(columns=["Profile Text Color"], inplace=True)

In [995]:
df_train['Profile Page Color'] = df_train['Profile Page Color'].apply(lambda x: x if(len(str(x)) == 6) else np.nan)
df_train['Profile Page Color'] = df_train['Profile Page Color'].apply(lambda x: hex2decimal(x))
new_cols = df_train['Profile Page Color'].str.split("|", expand = True)

df_train.insert(df_train.columns.get_loc("Profile Page Color"), "Profile Page Color R", new_cols[0])
df_train.insert(df_train.columns.get_loc("Profile Page Color"), "Profile Page Color G", new_cols[1])
df_train.insert(df_train.columns.get_loc("Profile Page Color"), "Profile Page Color B", new_cols[2])

df_train.drop(columns=["Profile Page Color"], inplace=True)

In [996]:
df_train['Profile Theme Color'] = df_train['Profile Theme Color'].apply(lambda x: x if(len(str(x)) == 6) else np.nan)
df_train['Profile Theme Color'] = df_train['Profile Theme Color'].apply(lambda x: hex2decimal(x))
new_cols = df_train['Profile Theme Color'].str.split("|", expand = True)

df_train.insert(df_train.columns.get_loc("Profile Theme Color"), "Profile Theme Color R", new_cols[0])
df_train.insert(df_train.columns.get_loc("Profile Theme Color"), "Profile Theme Color G", new_cols[1])
df_train.insert(df_train.columns.get_loc("Profile Theme Color"), "Profile Theme Color B", new_cols[2])

df_train.drop(columns=["Profile Theme Color"], inplace=True)

**Change unkown values in columns to NAN**


In [997]:
df_train["Profile Cover Image Status"].replace(2, np.nan, inplace=True)
df_train["Location Public Visibility"].replace(0, np.nan, inplace=True)


**Rounding Daily Profile Visits and Clicks**

In [998]:
df_train["Avg Daily Profile Visit Duration in seconds"] = df_train["Avg Daily Profile Visit Duration in seconds"].round()
df_train["Avg Daily Profile Clicks"] = df_train["Avg Daily Profile Clicks"].round()

**Filling NAN values with appropriate replacement (Median)**

In [999]:
df_train = df_train.fillna(df_train.median())

**Outlier deletion**

In [1000]:
def delete_outliers(df, name_of_column):
    Q1 = df.quantile(0.25)
    Q3 = df.quantile(0.75)
    IQR = Q3 - Q1

    mask = ((df > (Q1 - 1.5 * IQR)) & (df < (Q3 + 1.5 * IQR)))
    return df[mask[name_of_column]]

def correct_ouliers(df, name_of_column):
    upper_lim = df[name_of_column].quantile(.95)
    lower_lim = df[name_of_column].quantile(.05)

    df.loc[(df[name_of_column] > upper_lim), name_of_column] = upper_lim
    df.loc[(df[name_of_column] < lower_lim), name_of_column] = lower_lim
    return df

if generateTrain :
  continuous_columns = ["Num of Followers", "Num of People Following", "Num of Status Updates", "Num of Direct Messages", "Avg Daily Profile Visit Duration in seconds", "Avg Daily Profile Clicks", "Num of Profile Likes"]
  for c in continuous_columns:
      df_train = delete_outliers(df_train, c)

**Normalizing continuous values using log10**

In [1001]:
if generateTrain :
  continuous_columns = ["Num of Followers", "Num of People Following", "Num of Status Updates", "Num of Direct Messages", "Avg Daily Profile Visit Duration in seconds", "Avg Daily Profile Clicks", "Num of Profile Likes"]

if generateTest :
  continuous_columns = ["Num of Followers", "Num of People Following", "Num of Status Updates", "Num of Direct Messages", "Avg Daily Profile Visit Duration in seconds", "Avg Daily Profile Clicks"]

for c in continuous_columns:
  df_train[c] = np.log10(df_train[c] + 1)

**Convert Columns to appropriate type**

In [1002]:

df_train = df_train.astype({'Profile Cover Image Status': 'int64', 
                 'Personal URL': 'int64',
                 'UTC Offset': 'int64',
                 'Location Public Visibility': 'int64',
                 'Profile Creation WeekDay': 'int64',
                 "Profile Text Color R": 'float64',
                 "Profile Text Color G": 'float64',
                 "Profile Text Color B": 'float64',
                 "Profile Page Color R": 'float64',
                 "Profile Page Color G": 'float64',
                 "Profile Page Color B": 'float64',
                 "Profile Theme Color R": 'float64',
                 "Profile Theme Color G": 'float64',
                 "Profile Theme Color B": 'float64'
                 })


In [1003]:
encoded_columns = pd.get_dummies(df_train['Profile Creation Year'])
df_train = df_train.join(encoded_columns).drop('Profile Creation Year', axis=1)

df_train['Location Public Visibility'] = df_train['Location Public Visibility'].apply(lambda x: 0 if(x==1) else 1)

encoded_columns = pd.get_dummies(df_train['UTC Offset'])
df_train = df_train.join(encoded_columns).drop('UTC Offset', axis=1)
encoded_columns = pd.get_dummies(df_train['Profile Verification Status'])
df_train = df_train.join(encoded_columns).drop('Profile Verification Status', axis=1)
encoded_columns = pd.get_dummies(df_train['Profile Category'])
df_train = df_train.join(encoded_columns).drop('Profile Category', axis=1)


if generateTrain :
  tmp = df_train['Num of Profile Likes']
  df_train = df_train.drop('Num of Profile Likes', axis=1)
  df_train = df_train.join(tmp)

**Saving Model to CSV and Pkl**

In [1004]:
df_train

,Id,Personal URL,Profile Cover Image Status,Profile Text Color R,Profile Text Color G,Profile Text Color B,Profile Page Color R,Profile Page Color G,Profile Page Color B,Profile Theme Color R,Profile Theme Color G,Profile Theme Color B,Is Profile View Size Customized?,Location Public Visibility,Profile Creation Month,Profile Creation Day,Profile Creation WeekDay,Profile Creation Hour,User Language,Num of Followers,Num of People Following,Num of Status Updates,Num of Direct Messages,Avg Daily Profile Visit Duration in seconds,Avg Daily Profile Clicks,Profile Image,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,-39600,-36000,-28800,-25200,-21600,-18000,-14400,-10800,-7200,0,3600,7200,10800,14400,16200,18000,19800,21600,25200,28800,32400,36000,37800,39600,46800,Not verified,Pending,Verified,business,celebrity,government,unknown,Num of Profile Likes
2,JS49LP5P72RI1OQB,0,1,0.12,0.76,0.87,0.94,0.94,0.94,0.12,0.76,0.87,0,1,10,2,4,20,6,4.128561,3.273464,3.672005,2.357935,1.518514,0.301030,JS49LP5P72RI1OQB.png,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,3.193125
4,CRSEMK4QER6LDJSA,1,1,0.35,0.26,0.30,0.97,0.97,0.97,0.00,0.00,0.00,0,1,3,31,1,13,6,3.964542,1.973128,3.514813,2.117271,0.954243,0.698970,CRSEMK4QER6LDJSA.png,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,3.033021
5,0FEKBQ59AF1LQKJP,1,1,0.00,0.49,0.76,0.87,0.93,0.96,0.20,0.58,0.82,0,1,9,30,3,17,6,4.709160,2.372912,4.200905,2.671173,1.518514,0.301030,0FEKBQ59AF1LQKJP.png,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,2.230449
8,ORJN6I5G6NA6QD0Q,0,1,0.96,0.67,0.71,0.88,0.88,0.88,0.88,0.80,0.88,0,0,5,25,0,23,6,4.281283,2.149219,3.427811,2.320146,1.322219,0.845098,ORJN6I5G6NA6QD0Q.png,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,3.093071
9,5JIAFD6132MNPAL9,0,1,0.11,0.63,0.95,0.87,0.93,0.96,0.75,0.87,0.93,1,1,10,17,0,14,6,5.899709,2.869818,4.287174,2.380211,1.544068,0.602060,5JIAFD6132MNPAL9.png,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,2.722634
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7490,EQM27ASK6HGQHDTB,0,1,0.11,0.63,0.95,0.87,0.93,0.96,0.75,0.87,0.93,1,1,2,24,3,21,8,3.386677,2.996512,3.800648,1.662758,1.462398,0.477121,EQM27ASK6HGQHDTB.png,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,3.820661
7492,3QIOBBC2TCMCMOM8,0,1,0.00,0.52,0.71,0.87,0.93,0.96,0.75,0.87,0.93,0,0,8,1,0,6,6,4.893079,2.484300,3.926805,2.459392,1.531479,0.477121,3QIOBBC2TCMCMOM8.png,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,1,3.388101
7494,QSFDKHOHMHRNGS65,1,1,0.00,0.52,0.71,0.87,0.93,0.96,0.75,0.87,0.93,0,1,7,12,1,21,6,5.948095,2.290035,3.417638,3.085291,1.361728,0.301030,QSFDKHOHMHRNGS65.png,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,2.588832
7495,KP5DR7CTN8CNLG8P,0,0,0.29,0.03,0.03,0.87,0.93,0.96,0.00,0.00,0.00,0,0,1,20,3,20,6,3.784403,1.886491,2.993877,1.755875,1.414973,0.954243,KP5DR7CTN8CNLG8P.png,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,3.137987


In [1005]:
if generateTrain :
  df_train.to_pickle("train_cleaned_colors.pkl")
  df_train.to_csv("train_cleaned_colors.csv")

if generateTest :
  df_train.to_pickle("test_cleaned.pkl")
  df_train.to_csv("test_cleaned.csv")